# <center> BeakerX </center>

## <center> Volker Bäcker </center>

### <center> 27.01.2020 </center>

# <center> BeakerX features </center>

1. polyglot
1. EasyForm widgets
1. interactive plots
1. interactive tables
1. spark support
1. instant publication
1. java support

# <center> BeakerX history </center>

* beaker-notebook
> * independant notebook implementation

* beakerX
> * re-implementation as a jupyter-notebook extension

# <center> Polyglot - introduction </center>

* different cells in the same notebook can execute different languages
* data can be exchanged between languages

Data can be exchanged between languages by attaching it to the ``beakerx`` object.

In a ``groovy``-cell we attach the number 6 to the ``beakerx`` object with the name ``myValue``.

In [1]:
beakerx.myValue = 6

6

We get the value back in a ``java``-cell and modify it on the ``beakerx`` object.

In [2]:
%%java
Integer a = (Integer)NamespaceClient.getBeakerX().get("myValue");
System.out.println(a);
NamespaceClient.getBeakerX().set("myValue", a+1);

6


null

We get the current value in a ``kotlin``-cell. ``kotlin`` is a java like programming language prefered for ``android`` development.

In [3]:
%%kotlin
beakerx["myValue"]

7

In python it would work as well, but the OMERO-python kernel here is not configured to use beakerx <big>&#128577;</big>.

In [ ]:
%%python
from beakerx.object import beakerx
beakerx.myValue

# <center> Polyglot example </center>

* use python to download an image from omero
* use groovy to run an ImageJ-macro on it

We create a connetion-dialog using ``ipywidgets``. In ``beakerx`` we would use ``EasyForms`` which is easier to use.

Notes:
* We create widgets for the omero-host, the port, the username, the password, a connect button and a status text.
* The connect button has a function associated, which is executed when the button is pressed.
* The update of the status-text is not working correctly. The event-loop just does not get the time to do it. The status will be updated correctly after you executed the next cell. There are solutions for this, but no time now.... You can fix this as a homework <big>&#x1F642;</big>.

In [1]:
%%python
import ipywidgets as widgets
from omero.gateway import BlitzGateway
hosts = ["workshop.openmicroscopy.org", "omero.mri.cnrs.fr"]
port = 4064
CD_host = widgets.Dropdown(
       options=hosts,
       value=hosts[0],
       description='host:')
CD_port = widgets.IntText(
    value=port,
    description='port:',
) 
CD_top = widgets.HBox([CD_host, CD_port])
CD_user = widgets.Text(
    value = 'user',
    description = 'user:'
)
CD_password = widgets.Password(
    value = '',
    description = 'password:'
)
CD_button = widgets.Button(
    description = 'Connect',
)
CD_status = widgets.Label(
    description = "Status:",
    value = "Status: Not connected"
)
def on_connect_button_clicked(b):
    global conn
    CD_status.value = "Status: Connecting"
    conn = BlitzGateway(CD_user.value,
                        CD_password.value,
                        host=CD_host.value, port=CD_port.value)
    result = conn.connect()
    if result:
        CD_status.value = "Status: Connected"
    else:
        CD_status.value = "Status: Not connected"
    
CD_button.on_click(on_connect_button_clicked)
CD_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%')
CD = widgets.VBox([CD_top, widgets.HBox([CD_user, CD_password]), widgets.HBox([CD_button],  layout=CD_layout), widgets.HBox([CD_status], layout=CD_layout)])
display(CD)

In [ ]:
%%python
from IPython.display import Image
from PIL import Image as pImage
from io import BytesIO, StringIO
from base64 import b64encode

def on_project_change(change):
    global datasets
    datasets = list(projects[change['new']].listChildren())    
    BD_dataset.options = [(dataset.getName(), dataset.getId()) for dataset in datasets]
    BD_dataset.value =  BD_dataset.options[0][1]

def on_dataset_change(change):
    global images
    images = list(datasets[change['new']].listChildren())
    BD_image.options = [(image.getName(), image.getId()) for image in images] 
    BD_image.value = BD_image.options[0][1]

def on_image_change(change):
    global selectedImageIndex 
    selectedImageIndex = change['new']
    image = images[change['new']]
    img_data = image.getThumbnail()
    pil_im = pImage.open(BytesIO(img_data))
    img_bytes = BytesIO()  
    pil_im.save(img_bytes, format='png')
    BD_thumb.value = "<img src='data:image/png;base64,{0}' width=200, height=200/>".format(b64encode(img_bytes.getvalue()).decode('utf-8'))

projects = list(conn.getObjects("Project",
                               opts={'owner': conn.getUser().getId(),
                                     'group': conn.getEventContext().groupId,
                                     'order_by': 'lower(obj.name)'}))
projectTupels = [(project.getName(), project.getId()) for project in projects]

BD_project = widgets.Dropdown(
       continuous_update=True,
       options=projectTupels,
       value=projectTupels[0][1],
       description='project:')

BD_dataset = widgets.Dropdown(
       description='dataset:')

BD_image = widgets.Select(
       description='image:') 

BD_thumb = widgets.HTML("") 

on_project_change({'new':0})
on_dataset_change({'new':0})
on_image_change({'new':0})

BD_left = widgets.VBox([BD_project, BD_image])
BD_right = widgets.VBox([BD_dataset, BD_thumb, CD_layout])
BD_right.layout.align_items = 'flex-end'
BD_all = widgets.HBox([BD_left, BD_right])
BD_project.observe(on_project_change, names="index")
BD_dataset.observe(on_dataset_change, names="index")
BD_image.observe(on_image_change, names="index")
display(BD_all)


# <center> Polyglot example: download the images of the selected dataset </center>

In [ ]:
%%python
!rm -rf ./tmp
!mkdir -p ./tmp

import threading, time

DD_download_button = widgets.Button(
    description = 'Download dataset' 
)
DD_status = widgets.Label(
    description = "status",
    value = "Status: waiting..."
)

def on_download_button_clicked(b): 
    counter = 1
    for image in images:
        newStatus = "Status: downloading image " + str(counter) + " of " + str(len(images))
        DD_status.value = newStatus
        pixels = image.getPrimaryPixels()
        plane = pixels.getPlane()
        nucleiImage = pImage.fromarray(plane)
        nucleiImage.save('./tmp/'+image.getName(), save_all=True)
        counter = counter + 1
    DD_status.value = "Status: successfully downloaded " + str(counter-1) + " images."
DD_download_button.on_click(on_download_button_clicked)

DD = widgets.VBox([DD_download_button, DD_status])
display(DD)

In [ ]:
%%python
!ls -alh ./tmp

In [ ]:
%%python
%%writefile /home/jovyan/notebooks/volker/tmp/macro.ijm
SCALE_FACTOR = 5;
arg = getArgument();
parts = split(arg, ",");

for(i=0; i<parts.length; i++) {
	nameAndValue = split(parts[i], "=");
	if (indexOf(nameAndValue[0], "input")>-1) inputPath=nameAndValue[1];
	if (indexOf(nameAndValue[0], "scale_factor")>-1) SCALE_FACTOR=nameAndValue[1];
}
open(inputPath);
title = getTitle();
run("Set Measurements...", "area mean modal min centroid perimeter bounding fit shape feret's integrated median skewness kurtosis area_fraction display redirect="+title+" decimal=9");
width = getWidth();
height = getHeight();
run("Scale...", "x="+(1.0/SCALE_FACTOR)+" y="+(1.0/SCALE_FACTOR)+" interpolation=Bilinear create title=small_tmp");
setAutoThreshold("Huang dark");
run("Convert to Mask");
run("Fill Holes");
run("Watershed");
run("Scale...", "x=- y=- width="+width+" height="+height+" interpolation=Bilinear create title=big_tmp");
setAutoThreshold("Huang dark");
roiManager("Reset");
run("Analyze Particles...", "size=0-Infinity circularity=0.00-1.00 show=Overlay exclude display in_situ");
Overlay.copy;
close();
close();
Overlay.paste;
if (!File.exists(File.directory + "/out")) File.makeDirectory(File.directory + "/out");
save(File.directory + "/out/" + File.name);
run("Close All");
saveAs("results", File.directory + "/out/results.csv");
print("Done!");

In [ ]:
//Add dependencies to the classpath
%classpath add jar /opt/java-apps/Fiji.app/jars/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/*

In [ ]:
import ij.IJ
import static groovy.io.FileType.FILES
scaleFactor = 5

macro = new File('/home/jovyan/notebooks/volker/tmp/macro.ijm').text

def dir = new File("/home/jovyan/notebooks/volker/tmp");
def files = [];
dir.traverse(type: FILES, maxDepth: 0) { if (it.getName().contains(".TIF")) files.add(it) };
file = files[0]
args = "input="+file.getPath()+", scale_factor="+scaleFactor
IJ.runMacro('run("Clear Results");', "")
answer = IJ.runMacro(macro, args)

In [ ]:
%%python
!ls ./tmp/out

In [ ]:
csv = new CSV().read("/home/jovyan/notebooks/volker/tmp/out/results.csv")

Access a value in the table

In [ ]:
csv[3]["Area"]

Access a row in the table

In [ ]:
csv[3]

Access a column in the table

In [ ]:
csv["Area"]

# Plots and histograms

Create more data, by applying the macro to all images in the dataset.

In [ ]:
import ij.IJ
import static groovy.io.FileType.FILES
scaleFactor = 5

macro = new File('/home/jovyan/notebooks/volker/tmp/macro.ijm').text

def dir = new File("/home/jovyan/notebooks/volker/tmp");
def files = [];
dir.traverse(type: FILES, maxDepth: 0) { if (it.getName().contains(".TIF")) files.add(it) };
IJ.runMacro('run("Clear Results");', "")
files.each {
    file = it
    args = "input="+file.getPath()+", scale_factor="+scaleFactor
    answer = IJ.runMacro(macro, args)
}

In [ ]:
csv = new CSV().read("/home/jovyan/notebooks/volker/tmp/out/results.csv")

Creating a histogram

In [ ]:
areas = csv["Area"]
new Histogram(data: areas, binCount: 10);

Scatterplot: Are are and mean intensity correlated?

In [ ]:
intDens = csv['Mean']
def c = new Color(120, 120, 120, 100)
new Plot() << new Points(x: areas, y: intDens, displayName: "area vs mean")

In [ ]:
Is the mean area of nuclei different depending on the image?

In [ ]:
tupels = [csv['Label'],csv['Area']].transpose()
groups = tupels.groupBy {it[0]}.collectEntries {[(it.key): it.value.sum {it[1]}/it.value.size()]}
keys = groups.keySet()
values = groups.each { "$it.key"(it.value) } 
print(keys)
print(values)
new CategoryPlot(categoryMargin: 2) << new CategoryBars(value: [[1,2,3,4,5,6], groups.values()])

In [ ]:
%%python
conn.close()